## Install transfomers library and other required libraries / models

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.7MB/s 
     |████████████████████████████████| 3.3MB 37.1MB/s 
     |████████████████████████████████| 901kB 52.3MB/s 


In [ ]:
import torch
print(torch.__version__)
device = torch.device('cuda')

1.8.1+cu101


In [ ]:
torch.cuda.device(device)

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 454.3MB 40kB/s 


In [ ]:
 ! nvidia-smi

Thu May 27 15:34:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# without gpu use
# !pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



## Import RobertaArgs Pre-trained Model and Tokenizer
Reference:https://huggingface.co/chkla/roberta-argum

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("chkla/roberta-argument")
model = AutoModelForSequenceClassification.from_pretrained("chkla/roberta-argument")

* BERT-base model that has 110 million parameters

In [ ]:
model.config

RobertaConfig {
  "_name_or_path": "chkla/roberta-argument",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NON-ARGUMENT",
    "1": "ARGUMENT"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "ARGUMENT": 1,
    "NON-ARGUMENT": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.6.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

# Test on the Snippets data

In [ ]:
text_list =["In 2011 there were about 730,322 abortions reported to the centers for disease control.",
  "There are about 1.7% of abortion of women's ages from 15-44 each year.",
  "Women who already had abortion earlier in there life time have abortion again.",
  "At the age of 45 a women will have at least one abortion.",
  "By the 12th week of pregnancies 88.7% of women have abortion.",
  "In the U.S. black women are 3.3 times likely to have an abortion than white women.",
  "Abortion is wrong!",
  "Abortion Is gross!",
  "Abortion is MURDER!!!!"
]

## Tokenize and encode sequences of test set batch

In [ ]:
tokenized_batch = tokenizer(
    text_list,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

* Input ids:  dictionary of two items(sentences). Integer sequences of the input sentences.
*  Special tokens: integers 101 and 102 
*  Padding token : 0
*  Attention_mask: list of 1’s and 0’s. 
* Informs the model to pay attention to the tokens corresponding to the mask value of 1 and ignore the rest.

In [ ]:
tokenized_batch

{'input_ids': tensor([[    0,  1121,  1466,    89,    58,    59,   262,   541,     6, 34411,
         17600,   431,     7,     5,  5228,    13,  2199,   797,     4,     2,
             1,     1,     1,     1],
        [    0,   970,    32,    59,   112,     4,   406,   207,     9,  6428,
             9,   390,    18,  4864,    31,   379,    12,  3305,   349,    76,
             4,     2,     1,     1],
        [    0, 19814,    54,   416,    56,  6428,   656,    11,    89,   301,
            86,    33,  6428,   456,     4,     2,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    0,  3750,     5,  1046,     9,  2248,    10,   390,    40,    33,
            23,   513,    65,  6428,     4,     2,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    0,  2765,     5,   316,   212,   186,     9, 28303,  7953,     4,
           406,   207,     9,   390,    33,  6428,     4,     2,     1,     1,
             1,     1,     1,     1],
      

## Prediction on the pretrained model (binary-classification)

  
    



*   0 : NON-ARGUMENT
*   1 : ARGUMENT










    

In [ ]:
import torch.nn.functional as F
with torch.cuda.device(device):
  pt_outputs = model(**tokenized_batch)
  pt_predictions = F.softmax(pt_outputs.logits, dim=-1)
  pt_predictions_list = list(pt_predictions.detach().numpy())
  

pt_predictions_list

[array([0.848673  , 0.15132698], dtype=float32),
 array([0.84517086, 0.15482916], dtype=float32),
 array([0.17910108, 0.8208989 ], dtype=float32),
 array([0.7014524 , 0.29854763], dtype=float32),
 array([0.6685445 , 0.33145553], dtype=float32),
 array([0.5838554 , 0.41614464], dtype=float32),
 array([0.12000317, 0.8799969 ], dtype=float32),
 array([0.2165034 , 0.78349656], dtype=float32),
 array([0.08485993, 0.9151401 ], dtype=float32)]

In [ ]:
prediction_argument = []
for pred in pt_predictions_list:
  prediction_argument.append(pred[1])
prediction_argument

[0.15132698,
 0.15482916,
 0.8208989,
 0.29854763,
 0.33145553,
 0.41614464,
 0.8799969,
 0.78349656,
 0.9151401]

In [ ]:
import pandas as pd
Classification_argumentative_RobertArg = pd.DataFrame(
      
    {"Sentences" : text_list,
    "Predicted_probability": prediction_argument}
)

In [ ]:
Classification_argumentative_RobertArg

,Sentence,Predicted_probability
0,"In 2011 there were about 730,322 abortions rep...",0.151327
1,There are about 1.7% of abortion of women's ag...,0.154829
2,Women who already had abortion earlier in ther...,0.820899
3,At the age of 45 a women will have at least on...,0.298548
4,By the 12th week of pregnancies 88.7% of women...,0.331456
5,In the U.S. black women are 3.3 times likely t...,0.416145
6,Abortion is wrong!,0.879997
7,Abortion Is gross!,0.783497
8,Abortion is MURDER!!!!,0.915140


In [ ]:
Classification_argumentative_RobertArg.to_csv('Classification_argumentative_RobertArg.csv')